# LaAlO$_3$ - analysis of octahedral distortion with VanVleckCalculator

Here, we use data from the following paper:

Hayward, S. A., et al. "Transformation processes in LaAlO$_3$: Neutron diffraction, dielectric, thermal, optical, and Raman studies." Physical Review B 72.5 (2005): 054110.

In [ ]:
import pymatgen as pymatgen
from pymatgen.core.structure import Structure
import os
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
import sys
sys.path.insert(1,r"..\code")
from van_vleck_calculator import *

## Import structures from CIFs

In [ ]:
filenames = [i for i in os.listdir() if ".cif" in i]

In [ ]:
def obtain_temp_from_filename(filename):
    main_name = filename[:-4]
    temp_str = main_name.replace("cubic","").replace("_","").replace("K","").replace("rhom","")
    return(float(temp_str))

In [ ]:
temperatures = [obtain_temp_from_filename(filename) for filename in filenames]

In [ ]:
filenames = [x for _, x in sorted(zip(temperatures, filenames))]
temperatures = sorted(temperatures)

In [ ]:
structures = [Structure.from_file(i) for i in filenames]

## Create octahedra

In [ ]:
octahedra_objects = []
for i in range(0,len(structures)):
    sites = [j for j in structures[i] if "Al" in str(j.specie)]
    site = sites[0]
    local_oct = Octahedron(site,structures[i],possible_ligands=["O"])
    octahedra_objects.append(local_oct)

## Analyse data

In [ ]:
octahedral_volume = []

Q2_from_Al = []
Q3_from_Al = []
Q4_from_Al = []
Q5_from_Al = []
Q6_from_Al = []

Q2_from_av_pos = []
Q3_from_av_pos = []
Q4_from_av_pos = []
Q5_from_av_pos = []
Q6_from_av_pos = []

rho_from_Al = []
rho_from_av_pos = []

BLDI_from_Al = []
BLDI_from_av_pos = []

ECoN = []
QuadElon = []
BAV = []

pJT_dist = []
pJT_metric = []

delta_shear = []
delta_antishear = []
shear_fraction_from_Al = []
shear_fraction_from_av_pos = []

In [ ]:
for i in range(0,len(octahedra_objects)):
    octahedron = octahedra_objects[i]
    
    octahedral_volume.append(octahedron.volume)
    
    modes_from_Al = octahedron.calculate_van_vleck_distortion_modes(octahedral_centre="core_atom")
    modes_av_pos = octahedron.calculate_van_vleck_distortion_modes(octahedral_centre="average_ligand_position")
    
    Q2_from_Al.append(modes_from_Al[1])
    Q3_from_Al.append(modes_from_Al[2])
    Q4_from_Al.append(modes_from_Al[3])
    Q5_from_Al.append(modes_from_Al[4])
    Q6_from_Al.append(modes_from_Al[5])
    
    Q2_from_av_pos.append(modes_av_pos[1])
    Q3_from_av_pos.append(modes_av_pos[2])
    Q4_from_av_pos.append(modes_av_pos[3])
    Q5_from_av_pos.append(modes_av_pos[4])
    Q6_from_av_pos.append(modes_av_pos[5])
    
    rho_from_Al.append(
        octahedron.calculate_van_vleck_jahn_teller_params(octahedral_centre="core_atom")[0]
    )
    rho_from_av_pos.append(
        octahedron.calculate_van_vleck_jahn_teller_params(octahedral_centre="average_ligand_position")[0]
    )
    
    BLDI_from_Al.append(
        octahedron.calculate_bond_length_distortion_index(octahedral_centre="core_atom")
    )
    BLDI_from_av_pos.append(
        octahedron.calculate_bond_length_distortion_index(octahedral_centre="average_ligand_position")
    )

    ECoN.append(octahedron.calculate_effective_coordination_number())
    QuadElon.append(octahedron.calculate_quadratic_elongation())
    BAV.append(octahedron.calculate_bond_angle_variance(degrees=True))

    pJT_dist.append(octahedron.calculate_off_centering_distance())
    pJT_metric.append(octahedron.calculate_off_centering_metric())
    
    delta_shear.append(
        octahedron.calculate_angular_shear_magnitude(octahedral_centre="average_ligand_position")
    )
    
    delta_antishear.append(
        octahedron.calculate_angular_antishear_magnitude(octahedral_centre="average_ligand_position")
    )

    shear_fraction_from_Al.append(
        octahedron.calculate_shear_fraction_angular_distortion(octahedral_centre="core_atom")
    )
    shear_fraction_from_av_pos.append(
        octahedron.calculate_shear_fraction_angular_distortion(octahedral_centre="average_ligand_position")
    )

In [ ]:
df = pd.DataFrame(
    {
        "Temperature (K)":temperatures,
        "Octahedral volume (Å^3)":octahedral_volume,
#         "BLDI from Al":BLDI_from_Al,
        "BLDI from av. pos.":BLDI_from_av_pos,
        "Effective coordination number":ECoN,
        "Quadratic elongation":QuadElon,
        "Bond angle variance (°^2)":BAV,
#         "Q2 from Al (Å)":Q2_from_Al,
        "Q2 from av. pos. (Å)":Q2_from_av_pos,
#         "Q3 from Al (Å)":Q3_from_Al,
        "Q3 from av. pos. (Å)":Q3_from_av_pos,
#         "Q4 from Al (Å)":Q4_from_Al,
        "Q4 from av. pos. (Å)":Q4_from_av_pos,
#         "Q5 from Al (Å)":Q5_from_Al,
        "Q5 from av. pos. (Å)":Q5_from_av_pos,
#         "Q6 from Al (Å)":Q6_from_Al,
        "Q6 from av. pos. (Å)":Q6_from_av_pos,
#         "ρ from Al (Å)":rho_from_Al,
        "ρ from av. pos. (Å)":rho_from_av_pos,
#         "off-centering distance (Å)":pJT_dist,
#         "off-centering metric (Å)":pJT_metric,
#         "η from Al (Å)":shear_fraction_from_Al,
        "angular shear (°)":delta_shear,
        "angular antishear (°)":delta_antishear,
        "η from av. pos. (Å)":shear_fraction_from_av_pos,
    }
)

## Print table with Pandas

In [ ]:
df

In [ ]:
print(df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.3f}".format))